In [1]:
import cv2
import numpy as np
import time

In [2]:
# setup camera
cam = cv2.VideoCapture(0)

In [3]:
# tải mô hình đã dc huấn luyện
net = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt','./models/res10_300x300_ssd_iter_140000_fp16.caffemodel')
# Initialize variables
frame_count = 0
start_time = time.time()
font = cv2.FONT_HERSHEY_PLAIN
    
while True:
    # Read frame from camera
    ret, frame = cam.read()
    if not ret:
        break

    # Get image dimensions (moved outside the face detection loop)
    h, w = frame.shape[:2]

    # Prepare input blob
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (102, 105, 113), swapRB=False)

    # Set input and run face detection
    net.setInput(blob)
    faces = net.forward()

    # Calculate FPS efficiently
    frame_count += 1
    elapsed_time = time.time() - start_time
    if elapsed_time > 0:
        fps = frame_count / elapsed_time
        fps_text = f'FPS: {fps:.2f}'
        cv2.putText(frame, fps_text, (10, 50), font, 2, (255, 255, 0), 2)

    # Iterate through detected faces
    for i in range(faces.shape[2]):
        confi = faces[0, 0, i, 2]  # Confidence score
        if confi >= 0.7:  # Adjusted confidence threshold for better accuracy
            # Compute bounding box coordinates
            sx = int(faces[0,0,i,3] * w)  # X_min * width
            sy = int(faces[0,0,i,4] * h)  # Y_min * height
            ex = int(faces[0,0,i,5] * w)  # X_max * width
            ey = int(faces[0,0,i,6] * h)  # Y_max * height

            # Draw rectangle and display confidence score
            cv2.rectangle(frame, (sx, sy), (ex, ey), (255, 255, 0), 1)
            text = f'{confi * 100:.2f}%'
            cv2.putText(frame, text, (sx, sy - 10), font, 2, (255, 0, 255), 1)

    cv2.imshow('Detect', frame)
    if cv2.waitKey(10) == ord('q'):
        break

# Cleanup resources
cam.release()
cv2.destroyAllWindows()